In [1]:
from DataReader import DataReaderChallangePolimi2017
import implicit
from CB_asy import CB_asymmetric_cosine
from CF_IB_asy import CF_IB_asymmetric_cosine
from CF_UB_asy import CF_UB_asymmetric_cosine
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,TFIDFRecommender)
from sklearn.preprocessing import normalize
import numpy as np
import scipy.sparse as sp
import pickle

evaluation = True
rebuild = False
dr = DataReaderChallangePolimi2017(evaluation=evaluation,rebuild=rebuild)

urm = dr.getURM_csr()
t_p = dr.target_playlists
t_t = dr.target_tracks
if evaluation:
    filesslim = "./DumpData/sim_bpr_10e_100k2.npz" # or "./DumpData/sim_bpr_10e_100k.npz"
else:
    filesslim = "./DumpData/sim_bpr_10e_100k_fulltrain.npz"
    

Date Reader Challange Polimi 2017
Loading data from ./DumpData/DataReaderTrainTestDump
Dataset train-test loaded


In [2]:
#track-album
icm = dr.getICM_csr(albums=True,artists=False,tags=False)
icm = normalize(icm,axis=0)
model = BM25Recommender(K=55)
model.fit(icm)
s = model.similarity
s.setdiag(0)
r_al = urm*s.T
r_al = normalize(r_al, axis=0)
if evaluation: print dr.evaluateMAP(r_al,verbose=False)
r_al = dr.reduceRM(r_al)

C:\Users\Simone\Anaconda2\lib\site-packages\scipy\sparse\compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


0.0711872222222


In [3]:
#track-artist
icm = dr.getICM_csr(albums=False,artists=True,tags=False)
icm = normalize(icm,axis=0)
model = BM25Recommender(K=55)
model.fit(icm)
s = model.similarity
s.setdiag(0)
r_ar = urm*s.T
r_ar = normalize(r_ar, axis=0)
if evaluation: print dr.evaluateMAP(r_ar,verbose=False)
r_ar = dr.reduceRM(r_ar)

0.0702706944444


In [4]:
#playtlist-album
icm_al = dr.getUCM_csr(albums=True,artists=False)
model = BM25Recommender(K=100)
model.fit(icm_al)
s = model.similarity
r_pal = s.T*urm
r_pal = normalize(r_pal, axis=0)
if evaluation: print dr.evaluateMAP(r_pal,verbose=False)
r_pal = dr.reduceRM(r_pal)

0.0715785


In [5]:
#playlist-artist
icm_ar = dr.getUCM_csr(albums=False,artists=True)
model = BM25Recommender(K=100)
model.fit(icm_ar)
s = model.similarity
r_par = s.T*urm
r_par = normalize(r_par, axis=0)
if evaluation: print dr.evaluateMAP(r_par,verbose=False)
r_par = dr.reduceRM(r_par)

0.0607021666667


In [6]:
#album and artist
icm = dr.getICM_csr(albums=True,artists=True,tags=False)
icm = normalize(icm,axis=0)
model = BM25Recommender(K=55)
model.fit(icm)
s = model.similarity
r_al_ar = urm*s.T #or s
r_al_ar = normalize(r_al_ar, axis=0)
if evaluation: print dr.evaluateMAP(r_al_ar,verbose=False)
r_al_ar = dr.reduceRM(r_al_ar)

0.0790083611111


In [7]:
#tags
icm = dr.getICM_csr(albums=False,artists=False,tags=True)
icm = normalize(icm,axis=1)   # little improvment
model = BM25Recommender(K = 15)
model.fit(icm)
s = model.similarity
r_tag = urm*s
r_tag = normalize(r_tag, axis=0)
if evaluation: print dr.evaluateMAP(r_tag, verbose = False)
r_tag = dr.reduceRM(r_tag)

0.0377463611111


In [8]:
#cf ub
model = BM25Recommender(K=60)
model.fit(urm)
s = model.similarity
r_cfub = s.T * urm
r_cfub = normalize(r_cfub, axis=1)
if evaluation: print dr.evaluateMAP(r_cfub, verbose = False)
r_cfub = dr.reduceRM(r_cfub)

0.0661010277778


In [9]:
#cf ib
model = BM25Recommender(K=100)
model.fit(urm.T)
s = model.similarity
r_cfib = urm * s.T
r_cfib = normalize(r_cfib, axis=0)
if evaluation: print dr.evaluateMAP(r_cfib, verbose = False)
r_cfib = dr.reduceRM(r_cfib)

0.0729845


In [10]:
r_bpr = dr.reduceRM(urm * dr.load_sparse_csr(filesslim))

if evaluation: print dr.evaluateMAP(r_bpr, verbose=False)

0.0636542777778


In [11]:
#############start with the mix#################

In [12]:
######### cb

In [13]:
r_cb1 = r_al_ar + 0.25*r_tag 
if evaluation: print dr.evaluateMAP(r_cb1, verbose = False)

0.0811120277778


In [14]:
#not used atm
#r_totcb = normalize(r_cb1,axis=1) + 0.2* r_al + 0.15 * r_ar
#if evaluation: print dr.evaluateMAP(r_totcb, verbose = False)

In [15]:
########## cf

In [16]:
#total cf
r_totcf = 1.25*r_cfib + r_cfub
if evaluation: print dr.evaluateMAP(r_totcf, verbose = False)

0.0784290277778


In [17]:
########## all

In [18]:
r_tot = r_cb1 + 1.80 * r_totcf
if evaluation: print dr.evaluateMAP(r_tot,verbose = False)

0.104172


In [19]:
#not used atm
#r_tot2 = r_tot + 5* r_bpr

#if evaluation: print dr.evaluateMAP(r_tot2,verbose = False)

In [20]:
######improve total a little

In [21]:
r_tot2 = r_tot + 0.20*r_pal + 0.15*r_par
if evaluation: print dr.evaluateMAP(r_tot2,verbose = False)

0.104216083333


In [22]:
r_tot3 = r_tot2 + 0.10*r_al + 0.10*r_ar
if evaluation: print dr.evaluateMAP(r_tot3,verbose = False)

0.104762111111


In [23]:
#titles
ucm_titles = dr.getTitlesUCM_csr()
ucm = normalize(ucm_titles,axis=0)
model = BM25Recommender(K=100)
model.fit(ucm)
s = model.similarity
r_titles = s.T*urm
r_titles = normalize(r_titles, axis=1)
if evaluation: print dr.evaluateMAP(r_titles,verbose=False)
r_titles = dr.reduceRM(r_titles)

0.0108685277778


In [24]:
#owners
ucm_owners = dr.getOwnersUCM_csr()
ucm = normalize(ucm_owners,axis=0)
model = BM25Recommender(K=150)
model.fit(ucm)
s = model.similarity
r_owners = s.T*urm
r_owners = normalize(r_owners, axis=0)
if evaluation: print dr.evaluateMAP(r_owners,verbose=False)
r_owners_al = dr.reduceRM(r_owners)

0.0224909166667


In [25]:
r_tot4 = r_tot3 + 0.725*r_owners
if evaluation: print dr.evaluateMAP(r_tot4,verbose = False)

0.109232083333


In [26]:
######xxx project --> :( not work

In [27]:
owners, n_owners = dr.getOwners()
n_tracks = dr.n_tracks

train = dr.train.T
d = {}
for row in train:
    p = row[0]
    t = row[1]
    if p in d:
        d[p].append(t)
    else:
        d[p] = []
        d[p].append(t)

xxx = []
for row in owners.T:
    o = row[1]
    p = row[0]
    if p not in d:
        continue #we don't have this playlist in train
    t_p = d[p]
    for t in t_p:
        xxx.append([o,t])
xxx = np.array(xxx)

In [77]:
#build xxx matrix

r,c,v = [],[],[]
for row in xxx:
    r.append(row[0])
    c.append(row[1])
    v.append(1)

xcm = sp.coo_matrix((v, (r, c)), shape=(n_owners, n_tracks ),dtype=np.float).tocsr().T
xcm.data = np.ones(len(xcm.data))

In [114]:
icm= normalize(xcm,axis=1)
model = BM25Recommender(K=100)
model.fit(icm)
s = model.similarity
s.setdiag(0)
r_x = urm*s.T
r_x = normalize(r_x, axis=1)
if evaluation: print dr.evaluateMAP(r_x,verbose=False)
r_x = dr.reduceRM(r_x)

0.0383581944444


In [115]:
r_tot5 = r_tot4 + 0.13*r_x
if evaluation: print dr.evaluateMAP(r_tot5,verbose = False)

0.109488888889


In [ ]:
############# end xxx project

In [ ]:
#############

In [23]:
rec = dr.getAllRecommendations(r_tot3)
dr.buildSubmissionFile(recommendation=rec, file_name="ci_si_prova_3_mix")

User 0 of 10000
User 2500 of 10000
User 5000 of 10000
User 7500 of 10000
Recommendations done
Creating submission file in: ./Submissions/ci_si_prova_3_mix
Done


In [ ]:
## actually not used

In [8]:
#collaborative filtering item based asym
m_cf_ib = CF_IB_asymmetric_cosine(urm, k=200, alpha=0.30, q=1, m=0)
s = m_cf_ib.s
r_cfib_asy = urm*s
if evaluation: print dr.evaluateMAP(r_cfib_asy, verbose=False)
# reduce matrix
r_cfib_asy_more = normalize(r_cfib,axis=1) #4.75
r_cfib_asy_less = normalize(r_cfib,axis=0)#2.75 #######axis 0 better in sum

r_cfib_asy = dr.reduceRM(r_cfib_asy)
r_cfib_asy_more = dr.reduceRM(r_cfib_asy_more)
r_cfib_asy_less = dr.reduceRM(r_cfib_asy_less)

Start asymmetric cosine item-based model..
Keep only k-similar item..
Building sparse matrix..
Start building rating matrix..
Normalizing rating matrix..
Model asymmetric cosine item-based done
0.0720835555556
